In [1]:
using CSV, DataFrames, Gadfly, Distributions, SparseArrays

include("grid.jl");
include("GMRF.jl");
include("gibbs.jl");

# Création de la grille à 9 cellules

Chaque cellule est représentée par un triplet $[\mu, \sigma, \xi]$ indiquant les vraie valeur des paramètres de la GEV à déterminer.

In [2]:
# Dimensions de la grille
m₁ = 3;
m₂ = 3;
# Nb total de cellules
m = m₁ * m₂
# Insuffisance de rang de la matrice de structure
r = 1;
# Hyperparamètres
κᵤ = 100.0;
# Matrice de structure
W = buildStructureMatrix(m₁, m₂);
# Variances instrumentales de Metropolis pour μ et ϕ
σ² = 0.0015;

In [3]:
grid_target = generateTargetGridV1(m₁=m₁, m₂=m₂, r=r, κᵤ=κᵤ, W=W, σ²=σ²);

# Génération de données

Le but est de générer des données sur chaque cellule à partir des paramètres de la GEV de la grille cible.

In [4]:
nobs = 1000
data = generateData(grid_target, nobs);

# Modèle 

In [17]:
include("gibbs.jl");

niter = 100
itMetropolis = 100
δ² = [1000, 0.2, 0.001]
κᵤ₀ = 10
u₀ = zeros(m)
μ₀ = zeros(m)
W = buildStructureMatrix(m₁, m₂);

κᵤ, u, μ = gibbs(niter, data, itMetropolis=itMetropolis, δ²=δ², κᵤ₀=κᵤ₀, u₀=u₀, μ₀=μ₀, W=W);


In [18]:
reshape(μ[:, end], 3, 3)

3×3 Matrix{Float64}:
 0.0079232   0.00597513  0.0263595
 0.00668478  0.00195074  0.00218087
 0.017951    0.00720644  0.00536396

In [7]:
grid_target[:, :, 1]

3×3 Matrix{Float64}:
 0.62147   0.647534  0.669844
 0.573305  0.669176  0.641178
 0.666454  0.570453  0.702602

In [16]:
include("metropolis.jl")

u = generateSpatialEffect(200, θ₀=zeros(m), m=m, r=r, κ=100, W=W, σ²=0.0015)

println(u)

f̃κᵤ(κᵤ::Real) = fcκᵤ(κᵤ, W=W, u=u)

metropolisUnivariate(2000, θ₀=10, δ²=5, f̃=f̃κᵤ, printAcc=true)[end]

[-0.17595907795099397, -0.2945729005760615, -0.3495982743986302, -0.17632810989187425, -0.23835865485455862, -0.18944837056093733, -0.16739640790755278, -0.13807879258372477, -0.25011881028119876]
Taux d'acceptation : 31.315657828914457%


6.799409093526267